# Stigma Cue Prediction -- Neural Networks

In [1]:
from keras.layers import BatchNormalization, Conv1D, Dense, Dropout, Embedding, Flatten, GlobalMaxPooling1D, Input, LeakyReLU, LSTM
from keras.losses import BinaryCrossentropy
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot, text_to_word_sequence, Tokenizer
from keras.models import Sequential
from keras.optimizers import Adam, Nadam
from keras.regularizers import l1_l2
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

## Data Preparation

In [2]:
data = pd.read_csv('../data/output/20210302T19:23:10_stigma.csv', index_col='ID')
data = data.replace(-1.0, np.NaN).dropna(how='all').fillna(0)
data

,Stig_c1,Stig_c2,Stig_c3,Stig_c4,Stig_c5,Challn_c1,Challn_c2,Challn_c3,Challn_c4,Challn_c5
ID,,,,,,,,,,
Submission lwgnzw,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Comment gph9dfy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Comment gpi5kze,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Submission lwgo4f,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Comment gph9zt7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
Submission lxe14g,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Submission lxe1p5,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Submission lxe5js,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
sub = pd.read_csv('../data/input/2021-03-02T19:23:10/new_submissions.csv', index_col='id')
com = pd.read_csv('../data/input/2021-03-02T19:23:10/new_comments.csv', index_col='id')

subids = data.index.str.extract(r'Submission (.*)').dropna()
comids = data.index.str.extract(r'Comment (.*)').dropna()

sub = sub.loc[subids[0],:].rename(columns={'selftext': 'body'})
sub.index = 'Submission ' + sub.index
com = com.loc[comids[0],:]
com.index = 'Comment ' + com.index

In [4]:
subcom = pd.concat([sub, com], axis=0).loc[data.index,:][['body']]
full_data = subcom.join(data).dropna()
full_data

,body,Stig_c1,Stig_c2,Stig_c3,Stig_c4,Stig_c5,Challn_c1,Challn_c2,Challn_c3,Challn_c4,Challn_c5
ID,,,,,,,,,,,
Submission lwgnzw,"I really want to die, I feel like there’s noth...",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Comment gph9dfy,Please do not do what ever you are about to do...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Comment gpi5kze,me and big pepper are here to talk to you,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Submission lwgo4f,I'm in year 11 but my entire high school atten...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Comment gph9zt7,if there is a downhill in life there always mu...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
Submission lxe14g,I went to the attic last night. There was a be...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Submission lxe1p5,I seriously need someome to talk to. I have pl...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Submission lxe5js,I heard this phrase in this show the other day...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


### Splitting and Binarizing

In [5]:
X = full_data.body
y = full_data.drop('body', axis=1)
y = y.apply(lambda r: list(y.columns[r.astype(bool)]), axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

mlb = MultiLabelBinarizer().fit([list(data.columns)])
y_train = mlb.transform(y_train)
y_test = mlb.transform(y_test)

print(f'y_train:\n{y_train}', end='\n\n')
print('MLB Classes:')
print(list(mlb.classes_), sep=' ')

y_train:
[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]]

MLB Classes:
['Challn_c1', 'Challn_c2', 'Challn_c3', 'Challn_c4', 'Challn_c5', 'Stig_c1', 'Stig_c2', 'Stig_c3', 'Stig_c4', 'Stig_c5']


In [6]:
max_length = max(X.str.split().map(len))
num_words = len(set(X.str.replace(r'[^a-zA-Z\s]', '').str.replace(r'\n', ' ').str.lower().str.split(' ').sum()))

print(f'Max Length: {max_length}\nNum Words:  {num_words}\n')
print(f'X: {X_train.shape}  | {X_test.shape}\ny: {y_train.shape} | {y_test.shape}')

Max Length: 1134
Num Words:  5053

X: (454,)  | (114,)
y: (454, 10) | (114, 10)


## Models

In [7]:
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.00001

def standard_sequential(model_name: str, input_shape, output_shape=len(mlb.classes_)):
    """Standardized model architecture for comparing techniques (eg, embeddings)"""
    model = Sequential(name=model_name)
    model.add(Input(shape=input_shape))
    # model.add(Embedding(input_dim=X_train_tokens.shape[1], output_dim=200))
    model.add(Dense(1000, activation='relu'))
    # model.add(Conv1D(32, 11, activation='relu'))
    model.add(Dense(500, activation='relu'))
    # model.add(Conv1D(64, 7, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=['accuracy']
    )

    return model


def run_model(model, Xtr, Xts, ytr=y_train, yts=y_test):
    """Fits and evaluates a given model, printing metrics and returning the history and predictions."""
    hist = model.fit(Xtr, ytr, epochs=EPOCHS, verbose=True, validation_data=(Xts, yts), batch_size=BATCH_SIZE)
    y_pred = sequential_tokenized.predict(Xts)

    print(sequential_tokenized.evaluate(Xts, yts))
    print(confusion_matrix(yts, y_pred))
    print(classification_report(yts, y_pred))

    return hist, y_pred

### Sequential -- Count Vectorized

In [8]:
counts = CountVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1,3))
counts.fit(X)
X_train_vec = counts.transform(X_train)
X_test_vec = counts.transform(X_test)

X_train_vec.sort_indices()
X_test_vec.sort_indices()

print(f'X: {X_train_vec.shape}  | {X_test_vec.shape}\ny: {y_train.shape} | {y_test.shape}')

X: (454, 53167)  | (114, 53167)
y: (454, 10) | (114, 10)


In [9]:
sequential_counts = Sequential(name='sequential_counts')
sequential_counts.add(Input(shape=X_train_vec.shape[1]))
# sequential_counts.add(Embedding(input_dim=X_train_tokens.shape[1], output_dim=200))
sequential_counts.add(Dense(1000))
sequential_counts.add(BatchNormalization())
sequential_counts.add(LeakyReLU())
# sequential_counts.add(Conv1D(32, 11, activation='relu'))
sequential_counts.add(Dense(500))
sequential_counts.add(BatchNormalization())
sequential_counts.add(LeakyReLU())
# sequential_counts.add(Conv1D(64, 7, activation='relu'))
sequential_counts.add(Dense(100))
sequential_counts.add(BatchNormalization())
sequential_counts.add(LeakyReLU())
sequential_counts.add(Dense(len(mlb.classes_), activation='sigmoid'))

sequential_counts.compile(
    loss=BinaryCrossentropy(from_logits=True),
    optimizer=Nadam(learning_rate=0.0001),
    metrics=['accuracy']
)

sequential_counts.summary()

Model: "sequential_counts"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              53168000  
_________________________________________________________________
batch_normalization (BatchNo (None, 1000)              4000      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               500500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)         

In [10]:
hist = sequential_counts.fit(X_train_vec, y_train, epochs=50, verbose=True, validation_data=(X_test_vec, y_test), batch_size=32)
y_pred = sequential_counts.predict(X_test_vec)

print(sequential_counts.evaluate(X_test_vec, y_test))
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Epoch 1/50
15/15 [==============================] - 13s 745ms/step - loss: 0.7913 - accuracy: 0.1013 - val_loss: 0.6952 - val_accuracy: 0.1053
Epoch 2/50
15/15 [==============================] - 10s 697ms/step - loss: 0.5661 - accuracy: 0.6370 - val_loss: 0.6926 - val_accuracy: 0.0789
Epoch 3/50
15/15 [==============================] - 10s 700ms/step - loss: 0.4706 - accuracy: 0.7007 - val_loss: 0.6862 - val_accuracy: 0.0877
Epoch 4/50
15/15 [==============================] - 11s 712ms/step - loss: 0.4252 - accuracy: 0.7225 - val_loss: 0.6784 - val_accuracy: 0.0965
Epoch 5/50
15/15 [==============================] - 11s 712ms/step - loss: 0.3996 - accuracy: 0.7812 - val_loss: 0.6703 - val_accuracy: 0.0877
Epoch 6/50
15/15 [==============================] - 11s 717ms/step - loss: 0.3746 - accuracy: 0.8012 - val_loss: 0.6604 - val_accuracy: 0.1053
Epoch 7/50
15/15 [==============================] - 11s 730ms/step - loss: 0.3434 - accuracy: 0.7796 - val_loss: 0.6511 - val_accuracy: 0.1228

### Sequential -- Tokenized

In [39]:
# X_train_tokens = X_train.apply(lambda r: text_to_word_sequence(r))
# X_test_tokens = X_test.apply(lambda r: text_to_word_sequence(r))

tokenizer = Tokenizer(num_words=num_words + 1)
tokenizer.fit_on_texts(X)

# X_train_tokens = tokenizer.texts_to_matrix(X_train, mode='count')
# X_test_tokens = tokenizer.texts_to_matrix(X_test, mode='count')

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_train_tokens = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')

X_test_tokens = tokenizer.texts_to_sequences(X_test)
X_test_tokens = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

print(f'X Train: {X_train_tokens.shape}')
print(f'X Test:  {X_test_tokens.shape}')

X Train: (454, 1134)
X Test:  (114, 1134)


In [40]:
len(tokenizer.word_index)

5220

In [41]:
sequential_tokenized = Sequential(name='sequential_tokenized')
sequential_tokenized.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=200, input_length=max_length))
sequential_tokenized.add(GlobalMaxPooling1D())
# sequential_tokenized.add(Dense(1000))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
# sequential_tokenized.add(Dense(500))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
sequential_tokenized.add(Dense(250))
sequential_tokenized.add(BatchNormalization())
sequential_tokenized.add(LeakyReLU())
# sequential_tokenized.add(Dense(100))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
# sequential_tokenized.add(Dense(50))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
sequential_tokenized.add(Dense(len(mlb.classes_), activation='sigmoid'))

sequential_tokenized.compile(
    loss=BinaryCrossentropy(from_logits=True),
    optimizer=Nadam(learning_rate=0.001),
    metrics=['accuracy']
)

sequential_tokenized.summary()

Model: "sequential_tokenized"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1134, 200)         1044200   
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 200)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 250)               50250     
_________________________________________________________________
batch_normalization_24 (Batc (None, 250)               1000      
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 250)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 10)                2510      
Total params: 1,097,960
Trainable params: 1,097,460
Non-trainable params: 500
__________________________________

In [42]:
hist = sequential_tokenized.fit(X_train_tokens, y_train, epochs=50, verbose=True, validation_data=(X_test_tokens, y_test), batch_size=24)
y_pred = sequential_tokenized.predict(X_test_tokens)

print(sequential_tokenized.evaluate(X_test_tokens, y_test))
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Epoch 1/50
19/19 [==============================] - 1s 39ms/step - loss: 0.6703 - accuracy: 0.1510 - val_loss: 0.6179 - val_accuracy: 0.0088
Epoch 2/50
19/19 [==============================] - 1s 32ms/step - loss: 0.4588 - accuracy: 0.4448 - val_loss: 0.5213 - val_accuracy: 0.2719
Epoch 3/50
19/19 [==============================] - 1s 33ms/step - loss: 0.2998 - accuracy: 0.5044 - val_loss: 0.4875 - val_accuracy: 0.2895
Epoch 4/50
19/19 [==============================] - 1s 33ms/step - loss: 0.2190 - accuracy: 0.6134 - val_loss: 0.4753 - val_accuracy: 0.3509
Epoch 5/50
19/19 [==============================] - 1s 33ms/step - loss: 0.1557 - accuracy: 0.6729 - val_loss: 0.4453 - val_accuracy: 0.3684
Epoch 6/50
19/19 [==============================] - 1s 32ms/step - loss: 0.1113 - accuracy: 0.7417 - val_loss: 0.4357 - val_accuracy: 0.3947
Epoch 7/50
19/19 [==============================] - 1s 32ms/step - loss: 0.0887 - accuracy: 0.7331 - val_loss: 0.4228 - val_accuracy: 0.4474
Epoch 8/50
19

In [46]:
sequential_tokenized_conv = Sequential(name='sequential_tokenized_conv')
# sequential_tokenized.add(Input(shape=X_train_tokens.shape[1]))
sequential_tokenized_conv.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=200, input_length=max_length))
sequential_tokenized_conv.add(Conv1D(256, 3, padding='valid'))
sequential_tokenized_conv.add(BatchNormalization())
sequential_tokenized_conv.add(LeakyReLU())
sequential_tokenized_conv.add(GlobalMaxPooling1D())
# sequential_tokenized.add(Conv1D(32, 11, activation='relu'))
# sequential_tokenized_conv.add(Dense(250))
# sequential_tokenized_conv.add(BatchNormalization())
# sequential_tokenized_conv.add(LeakyReLU())
# sequential_tokenized.add(Conv1D(64, 7, activation='relu'))
# sequential_tokenized_conv.add(Dense(100))
# sequential_tokenized_conv.add(BatchNormalization())
# sequential_tokenized_conv.add(LeakyReLU())
# sequential_tokenized.add(Dropout(.2))
sequential_tokenized_conv.add(Dense(len(mlb.classes_), activation='sigmoid'))

sequential_tokenized_conv.compile(
    loss=BinaryCrossentropy(from_logits=True),
    optimizer=Nadam(learning_rate=0.001),
    metrics=['accuracy']
)

sequential_tokenized_conv.summary()

Model: "sequential_tokenized_conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1134, 200)         1044200   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1132, 256)         153856    
_________________________________________________________________
batch_normalization_26 (Batc (None, 1132, 256)         1024      
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 1132, 256)         0         
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 10)                2570      
Total params: 1,201,650
Trainable params: 1,201,138
Non-trainable params: 512
_____________________________

In [47]:
hist = sequential_tokenized_conv.fit(X_train_tokens, y_train, epochs=50, verbose=True, validation_data=(X_test_tokens, y_test), batch_size=24)
y_pred = sequential_tokenized_conv.predict(X_test_tokens)

print(sequential_tokenized_conv.evaluate(X_test_tokens, y_test))
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Epoch 1/50
19/19 [==============================] - 5s 219ms/step - loss: 0.6294 - accuracy: 0.2054 - val_loss: 0.6869 - val_accuracy: 0.1842
Epoch 2/50
19/19 [==============================] - 4s 211ms/step - loss: 0.1872 - accuracy: 0.6537 - val_loss: 0.6735 - val_accuracy: 0.3246
Epoch 3/50
19/19 [==============================] - 4s 214ms/step - loss: 0.0845 - accuracy: 0.7779 - val_loss: 0.6606 - val_accuracy: 0.3860
Epoch 4/50
19/19 [==============================] - 4s 212ms/step - loss: 0.0486 - accuracy: 0.7764 - val_loss: 0.6437 - val_accuracy: 0.4211
Epoch 5/50
19/19 [==============================] - 4s 214ms/step - loss: 0.0321 - accuracy: 0.7533 - val_loss: 0.6290 - val_accuracy: 0.4211
Epoch 6/50
19/19 [==============================] - 4s 211ms/step - loss: 0.0212 - accuracy: 0.7612 - val_loss: 0.6131 - val_accuracy: 0.4386
Epoch 7/50
19/19 [==============================] - 4s 210ms/step - loss: 0.0163 - accuracy: 0.7587 - val_loss: 0.5975 - val_accuracy: 0.4386
Epoch 

In [ ]:
# sequential_tokenized_lstm = Sequential(name='sequential_tokenized_lstm')
# sequential_tokenized.add(Input(shape=X_train_tokens.shape[1]))
# sequential_tokenized_lstm.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
# sequential_tokenized.add(Flatten())
# sequential_tokenized_lstm.add(Conv1D(256, 7, padding='valid'))
# sequential_tokenized_lstm.add(BatchNormalization())
# sequential_tokenized_lstm.add(LeakyReLU())
# sequential_tokenized_lstm.add(Conv1D(512, 5))
# sequential_tokenized_lstm.add(BatchNormalization())
# sequential_tokenized_lstm.add(LeakyReLU())
# sequential_tokenized_lstm.add(GlobalMaxPooling1D())
# sequential_tokenized.add(Conv1D(32, 11, activation='relu'))
# sequential_tokenized_lstm.add(Dense(500))
# sequential_tokenized_lstm.add(BatchNormalization())
# sequential_tokenized_lstm.add(LeakyReLU())
# sequential_tokenized.add(Conv1D(64, 7, activation='relu'))
# sequential_tokenized_lstm.add(Dense(100))
# sequential_tokenized_lstm.add(BatchNormalization())
# sequential_tokenized_lstm.add(LeakyReLU())
# sequential_tokenized.add(Dropout(.2))
# sequential_tokenized_lstm.add(Dense(len(mlb.classes_), activation='sigmoid'))

# sequential_tokenized_lstm.compile(
#     loss=BinaryCrossentropy(from_logits=True),
#     optimizer=Nadam(learning_rate=LEARNING_RATE),
#     metrics=['accuracy']
# )

# sequential_tokenized_lstm.summary()

### Sequential -- TF-IDF

In [48]:
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii', ngram_range=(1,3))
tfidf.fit(X)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.sort_indices()
X_test_tfidf.sort_indices()

print(f'TF-IDF\nX: {X_train_tfidf.shape} | {X_test_tfidf.shape}\ny: {y_train.shape} | {y_test.shape}')

TF-IDF
X: (454, 53167) | (114, 53167)
y: (454, 10) | (114, 10)


In [52]:
len(tfidf.get_feature_names())

53167

In [68]:
sequential_tfidf = Sequential(name='sequential_tfidf')
sequential_tfidf.add(Input(shape=X_train_tfidf.shape[1]))
# sequential_tfidf.add(Embedding(input_dim=X_train_tokens.shape[1], output_dim=200))
# sequential_tfidf.add(Dense(1000))
# sequential_counts.add(BatchNormalization())
# sequential_counts.add(LeakyReLU())
# sequential_tfidf.add(Conv1D(32, 11, activation='relu'))
# sequential_tfidf.add(Dense(250))
# sequential_counts.add(BatchNormalization())
# sequential_counts.add(LeakyReLU())
# sequential_tfidf.add(Conv1D(64, 7, activation='relu'))
sequential_tfidf.add(Dense(50))
sequential_counts.add(BatchNormalization())
sequential_counts.add(LeakyReLU())
# sequential_tfidf.add(Dropout(.5))
sequential_tfidf.add(Dense(len(mlb.classes_), activation='sigmoid'))

sequential_tfidf.compile(
    loss=BinaryCrossentropy(from_logits=True),
    optimizer=Nadam(learning_rate=0.001),
    metrics=['accuracy']
)

sequential_tfidf.summary()

Model: "sequential_tfidf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 50)                2658400   
_________________________________________________________________
dense_51 (Dense)             (None, 10)                510       
Total params: 2,658,910
Trainable params: 2,658,910
Non-trainable params: 0
_________________________________________________________________


In [69]:
hist = sequential_tfidf.fit(X_train_tfidf, y_train, epochs=50, verbose=True, validation_data=(X_test_tfidf, y_test), batch_size=32)
y_pred = sequential_tfidf.predict(X_test_tfidf)

print(sequential_tfidf.evaluate(X_test_tfidf, y_test))
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Epoch 1/50
15/15 [==============================] - 1s 41ms/step - loss: 0.6855 - accuracy: 0.1346 - val_loss: 0.6563 - val_accuracy: 0.2895
Epoch 2/50
15/15 [==============================] - 1s 35ms/step - loss: 0.6204 - accuracy: 0.4697 - val_loss: 0.6002 - val_accuracy: 0.2807
Epoch 3/50
15/15 [==============================] - 1s 34ms/step - loss: 0.5308 - accuracy: 0.3098 - val_loss: 0.5406 - val_accuracy: 0.2719
Epoch 4/50
15/15 [==============================] - 0s 32ms/step - loss: 0.4432 - accuracy: 0.2695 - val_loss: 0.4923 - val_accuracy: 0.2807
Epoch 5/50
15/15 [==============================] - 0s 32ms/step - loss: 0.3799 - accuracy: 0.3178 - val_loss: 0.4589 - val_accuracy: 0.2807
Epoch 6/50
15/15 [==============================] - 0s 33ms/step - loss: 0.3325 - accuracy: 0.4048 - val_loss: 0.4370 - val_accuracy: 0.2807
Epoch 7/50
15/15 [==============================] - 0s 33ms/step - loss: 0.2920 - accuracy: 0.4728 - val_loss: 0.4217 - val_accuracy: 0.2807
Epoch 8/50
15

In [71]:
sequential_tfidf_emb = Sequential(name='sequential_tfidf_emb')
sequential_tfidf_emb.add(Embedding(input_dim=len(tfidf.vocabulary_) + 1, output_dim=200, input_length=max_length))
sequential_tfidf_emb.add(GlobalMaxPooling1D())
# sequential_tokenized.add(Dense(1000))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
# sequential_tokenized.add(Dense(500))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
sequential_tfidf_emb.add(Dense(250))
sequential_tfidf_emb.add(BatchNormalization())
sequential_tfidf_emb.add(LeakyReLU())
# sequential_tokenized.add(Dense(100))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
# sequential_tokenized.add(Dense(50))
# sequential_tokenized.add(BatchNormalization())
# sequential_tokenized.add(LeakyReLU())
sequential_tfidf_emb.add(Dense(len(mlb.classes_), activation='sigmoid'))

sequential_tfidf_emb.compile(
    loss=BinaryCrossentropy(from_logits=True),
    optimizer=Nadam(learning_rate=0.001),
    metrics=['accuracy']
)

sequential_tfidf_emb.summary()

Model: "sequential_tfidf_emb"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 1134, 200)         10633600  
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 200)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 250)               50250     
_________________________________________________________________
batch_normalization_40 (Batc (None, 250)               1000      
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 250)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 10)                2510      
Total params: 10,687,360
Trainable params: 10,686,860
Non-trainable params: 500
________________________________

In [72]:
hist = sequential_tfidf_emb.fit(X_train_tfidf, y_train, epochs=50, verbose=True, validation_data=(X_test_tfidf, y_test), batch_size=32)
y_pred = sequential_tfidf_emb.predict(X_test_tfidf)

print(sequential_tfidf_emb.evaluate(X_test_tfidf, y_test))
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

Epoch 1/50


TypeError: in user code:

    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/Michael/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:34